In [9]:
import pandas as pd
import numpy as np
import math
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error

# Нормировка признаков


In [40]:
def normalize(X,nrows):
    BinaryFeatures = ['Post Promotion Status','published_weekday_0','published_weekday_1','published_weekday_2',
                      'published_weekday_3', 'published_weekday_4','published_weekday_5','published_weekday_6',
                     'base_weekday_0','base_weekday_1','base_weekday_2','base_weekday_3','base_weekday_4',
                      'base_weekday_5','base_weekday_6']
    BF = X[BinaryFeatures] 
    Y = X['Target']
    DecimalFeatures = ['Page Popularity','Page Checkins','Page talking about',
                       'extra_0','extra_1','extra_2','extra_3','extra_4','extra_5',
                       'extra_6','extra_7','extra_8','extra_9','extra_10',
                       'extra_11','extra_12','extra_13','extra_14','extra_15','extra_16',
                       'extra_17','extra_18','extra_19','extra_20','extra_21','extra_22','extra_23','extra_24',
                       'CC1','CC2','CC3','CC4','CC5','Base Time','Post Length','Post Share Count','H Local']
    DF = X[DecimalFeatures]
    E = DF.mean()
    D = DF.std()
    DF = (DF-E)/D
    
    frequency = X['Page Category'].value_counts(sort=False) #frequency of categories
    CF = 1/frequency[X['Page Category']]
    CF = CF.reset_index(drop=True)
    
    # x0 = [1,1,...1] fpr b koeficient
    x1 = pd.DataFrame({'feature 1': np.ones(nrows)})


    result = pd.concat([x1,DF,CF, BF,Y], axis=1) #,X['H Local']
    return result



# Градиент, RMSE, R_2

In [6]:
def gradient(X,Y,W,lyambda, n, nfeat):
    Ypred = np.dot(X,W)
    #print(Y[:10], Ypred[:10])
    dy = Y - Ypred
    cost = np.sum(dy**2)/n
    #print( "cost", cost)
    gr = np.dot(X.T,dy)/n
    Wnew = W + 2* lyambda * gr.reshape(nfeat,1)
    return Wnew

def foo_RMSE(Xt, Yt, W):
    rmse = math.sqrt(np.sum(((Yt-np.dot(Xt,W))**2))/len(Yt))
    return rmse

def foo_R2(X,Y,W):
    Ey = Y.mean()
    r2 = 1 - np.sum((np.dot(X,W)-Y)**2) / np.sum((Y - Ey)**2)
    return r2

# Формирование X и Y

In [48]:
df = pd.read_csv('Dataset.csv', sep=',')
df = df.drop(df.columns[0],axis=1)
#X = df[df.columns[:53]]
#Y = df[['Target']]
data = df
data = normalize(data,data.shape[0])
#X = norm(X)
N =  X.shape[0]


In [72]:

#init   
nsteps = 400
e = 10e-4
lyambda = 0.05

table = pd.DataFrame({})
nfeatures = X.shape[1]
chunk = 0
nfolds = 5

for i in range(nfolds):
    
    #train/test split
    data = shuffle(data)
    Y = data[['Target']]
    X = data[data.columns[:54]]
   
    train_size = int(N * 0.6)
    
    Xtrain = X[:train_size]
    Ytrain = Y[:train_size]
    Xtest = X[train_size:]
    Ytest = Y[train_size:]  
    nrows = Xtrain.shape[0]
    chunk+=1
    k = 0
    W = np.ones(nfeatures).reshape(nfeatures,1)
    
    while True:
        k+=1
        Wnew = gradient(Xtrain,Ytrain,W,lyambda/math.sqrt(k), nrows, nfeatures)
        estimation = np.max(abs(W - Wnew))
    
        if (estimation < e)or(k>nsteps):
            break
        W = Wnew
        
    print("CHUNK # ", chunk)    
    R2train = foo_R2(Xtrain,Ytrain,W)
    print ("R2 for train: ",R2train[0] , "\n")

    RMSEtrain = foo_RMSE(Xtrain,Ytrain,W)
    print ("RMSE for train: ",RMSEtrain , "\n") 
 
    
    #test
    R2test = foo_R2(Xtest,Ytest,W)
    print ("R2 for test: ",R2test[0] , "\n")

    RMSEtest = foo_RMSE(Xtest,Ytest,W)
    print ("RMSE for test: ",RMSEtest , "\n")
   
    
    a = [R2train[0],RMSEtrain,R2test[0],RMSEtest]
    
    for feature in W.tolist():
        a.append(feature[0])
    col = pd.DataFrame({'chunk %d'%chunk: a })
    table =  pd.concat([table, col],axis = 1)


table.index=['R2_train', 'RMSE_train', 'R2_test', 'RMSE_test','b','Page Popularity','Page Checkins',
             'Page talking about','extra_0','extra_1','extra_2','extra_3','extra_4','extra_5','extra_6',
             'extra_7','extra_8','extra_9','extra_10','extra_11','extra_12','extra_13','extra_14','extra_15',
             'extra_16','extra_17','extra_18','extra_19','extra_20','extra_21','extra_22','extra_23','extra_24',
             'CC1','CC2','CC3','CC4','CC5','Base Time','Post Length','Post Share Count','H Local','Page Category', 
             'Post Promotion Status','published_weekday_0','published_weekday_1','published_weekday_2',
             'published_weekday_3', 'published_weekday_4','published_weekday_5','published_weekday_6',
             'base_weekday_0','base_weekday_1','base_weekday_2','base_weekday_3','base_weekday_4',
             'base_weekday_5','base_weekday_6']
table.index.name = 'Features'
table =  pd.concat([table, pd.DataFrame({'E': table.mean(axis = 1) })],axis = 1)
table =  pd.concat([table, pd.DataFrame({'STD': table.std(axis = 1) })],axis = 1)
table.to_csv('table.csv')   

CHUNK #  1
R2 for train:  0.31124044765 

RMSE for train:  29.163680679675643 

R2 for test:  0.316577222625 

RMSE for test:  29.775220764726768 

CHUNK #  2
R2 for train:  0.336479871536 

RMSE for train:  29.79458374191181 

R2 for test:  0.275409184093 

RMSE for test:  28.775027022264055 

CHUNK #  3
R2 for train:  0.320129389493 

RMSE for train:  27.631196024629503 

R2 for test:  0.30961829048 

RMSE for test:  31.802684344987238 

CHUNK #  4
R2 for train:  0.321797482107 

RMSE for train:  29.89356355880233 

R2 for test:  0.307684422319 

RMSE for test:  28.498481064533888 

CHUNK #  5
R2 for train:  0.318060718421 

RMSE for train:  28.397897715149078 

R2 for test:  0.312645565897 

RMSE for test:  30.75039212887015 



In [73]:
table

,chunk 1,chunk 2,chunk 3,chunk 4,chunk 5,E,STD
Features,,,,,,,
R2_train,0.311240,0.336480,0.320129,0.321797,0.318061,0.321542,0.008288
RMSE_train,29.163681,29.794584,27.631196,29.893564,28.397898,28.976184,0.859061
R2_test,0.316577,0.275409,0.309618,0.307684,0.312646,0.304387,0.014797
RMSE_test,29.775221,28.775027,31.802684,28.498481,30.750392,29.920361,1.231803
b,4.348740,4.397663,4.269436,4.365497,4.302357,4.336739,0.045595
Page Popularity,-0.902849,0.058864,-0.073938,-0.010963,0.014036,-0.182970,0.362490
Page Checkins,-0.436705,-0.711141,-0.655599,-0.608042,-0.602435,-0.602784,0.091805
Page talking about,-0.496212,-1.519744,-0.932019,-1.065962,-1.008162,-1.004420,0.326459
extra_0,-0.140906,0.290164,-0.512486,-0.237339,-0.027865,-0.125686,0.262607
